In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import random

In [4]:
with open('/content/input.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])

In [5]:
def map_tag_pattern(df, tag_col, text_col, res_col):
    tags = []
    inputs = []
    responses = {}

    for _, item in df.iterrows():
        ptrns = item[text_col]
        tag = item[tag_col]
        responses[item[tag_col]] = item[res_col]
        for j in range(len(ptrns)):
            tags.append(tag)
            inputs.append(ptrns[j])

    return tags, inputs, responses

# Assuming df is your original DataFrame
tags, inputs, responses = map_tag_pattern(df, "tag", "patterns", "responses")

# Example of how to use the lists
print("Tags:", tags)
print("Patterns:", inputs)
print("Responses:", responses)

Tags: ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'courtesy-greeting', 'courtesy-greeting', 'courtesy-greeting', 'courtesy-greeting', 'courtesy-greeting', 'courtesy-greeting', 'courtesy-greeting', 'morning', 'afternoon', 'evening', 'night', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'counter-goodbye', 'counter-goodbye', 'counter-goodbye', 'counter-goodbye', 'counter-goodbye', 'counter-goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'no-response', 'neutral-response', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'about', 'skill', 'creation', 'creation', 'creation', 'name', 'name', 'name', 'help', 'help', 'help', 'help', 'help', 'help', 'help', 'sad', 'sad', 'sad', 'sad', 'sad',

In [6]:
data = pd.DataFrame({"inputs":inputs, "tags":tags})
data

,inputs,tags
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hi there,greeting
4,Hello,greeting
...,...,...
261,I want to learn about mental health.,learn-mental-health
262,I want to learn more about mental health.,learn-mental-health
263,I'm interested in learning about mental health.,learn-mental-health
264,Tell me a fact about mental health,mental-health-fact


In [7]:
data["inputs"] = data["inputs"].apply(lambda wrd: ''.join([ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation]))

In [8]:
tokenizer = Tokenizer(num_words = 2000)
tokenizer.fit_on_texts(data["inputs"])
train = tokenizer.texts_to_sequences(data["inputs"])
le = LabelEncoder()

In [9]:
x_train = pad_sequences(train)
y_train = le.fit_transform(data["tags"])

In [10]:
# hyperparameters
epochs=500
input_shape = x_train.shape[1]
vocab_size = len(tokenizer.word_index) + 1
output_length = le.classes_.shape[0]
embed_dim=10

In [17]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(input_shape,)),
    tf.keras.layers.Embedding(vocab_size, embed_dim, input_length=(input_shape,)),
    tf.keras.layers.LSTM(units=10, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=10, activation='relu'),
    tf.keras.layers.Dense(units=5, activation='relu'),
    tf.keras.layers.Dense(output_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 18, 10)            2890      
                                                                 
 lstm_4 (LSTM)               (None, 18, 10)            840       
                                                                 
 flatten_3 (Flatten)         (None, 180)               0         
                                                                 
 dense_12 (Dense)            (None, 10)                1810      
                                                                 
 dense_13 (Dense)            (None, 5)                 55        
                                                                 
 dense_14 (Dense)            (None, 58)                348       
                                                                 
Total params: 5943 (23.21 KB)
Trainable params: 5943 (

In [18]:
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
# model.fit(x_train, y_train, epochs=epochs, callbacks=[early_stop])
model.fit(x_train, y_train, epochs=epochs)

Epoch 1/500
9/9 [==============================] - 3s 95ms/step - loss: 4.0600 - accuracy: 0.0188
Epoch 2/500
9/9 [==============================] - 1s 85ms/step - loss: 4.0569 - accuracy: 0.0789
Epoch 3/500
9/9 [==============================] - 0s 50ms/step - loss: 4.0544 - accuracy: 0.0752
Epoch 4/500
9/9 [==============================] - 1s 70ms/step - loss: 4.0513 - accuracy: 0.0752
Epoch 5/500
9/9 [==============================] - 1s 68ms/step - loss: 4.0463 - accuracy: 0.0263
Epoch 6/500
9/9 [==============================] - 0s 39ms/step - loss: 4.0345 - accuracy: 0.0188
Epoch 7/500
9/9 [==============================] - 0s 16ms/step - loss: 4.0219 - accuracy: 0.0188
Epoch 8/500
9/9 [==============================] - 0s 40ms/step - loss: 4.0080 - accuracy: 0.0188
Epoch 9/500
9/9 [==============================] - 0s 28ms/step - loss: 3.9933 - accuracy: 0.0188
Epoch 10/500
9/9 [==============================] - 0s 52ms/step - loss: 3.9798 - accuracy: 0.0188
Epoch 11/500
9/9 [=

In [19]:
while True:
    textList = []
    prediction_input = []
    user_input = input("Input (press 'q' to quit): ")
    if user_input.lower() == 'q':
        break
    else:
        for letter in user_input:
            if letter not in string.punctuation:
                prediction_input.append(letter.lower())

        prediction_input = ''.join(prediction_input)
        textList.append(prediction_input)

        prediction_input = tokenizer.texts_to_sequences(textList)
        prediction_input = np.array(prediction_input).reshape(-1)
        prediction_input = pad_sequences([prediction_input], input_shape)

        output = model.predict(prediction_input)
        output = output.argmax()

        response_tag = le.inverse_transform([output])[0]
        print("AI: ", random.choice(responses[response_tag]))

Input (press 'q' to quit): hi
1/1 [==============================] - 0s 390ms/step
AI:  Hello there. Glad to see you're back. What's going on in your world right now?
Input (press 'q' to quit): how are you
1/1 [==============================] - 0s 17ms/step
AI:  Hello, I am great, how are you? Please tell me your GeniSys user
Input (press 'q' to quit): good morning
1/1 [==============================] - 0s 18ms/step
AI:  Good morning. I hope you had a good night's sleep. How are you feeling today? 
Input (press 'q' to quit): q
